In [17]:
#import dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.regularizers import l2

In [18]:
#create df of preprocessed data - game data from 2004-2024
df = pd.read_csv('nbaPreProcessed.csv')
df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame,winner_binary
0,2024-06-17,Boston Celtics,BOS,2,0.792079,80,21,106,28,39,...,35,18,4,4,13,20,Boston Celtics,2023-2024,True,0
1,2024-06-14,Dallas Mavericks,DAL,7,0.611650,63,40,122,34,27,...,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True,0
2,2024-06-12,Dallas Mavericks,DAL,7,0.607843,62,40,99,31,20,...,36,26,4,6,9,19,Boston Celtics,2023-2024,True,1
3,2024-06-09,Boston Celtics,BOS,2,0.795918,78,20,105,25,29,...,43,21,5,3,15,17,Boston Celtics,2023-2024,True,0
4,2024-06-06,Boston Celtics,BOS,2,0.793814,77,20,107,37,26,...,43,9,8,1,11,16,Boston Celtics,2023-2024,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,1,104,21,19,...,47,24,9,5,13,28,Indiana Pacers,2004-2005,False,1
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,1,95,22,31,...,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False,1
25728,2004-11-02,Los Angeles Lakers,LAL,14,1.000000,1,0,89,24,26,...,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False,0
25729,2004-11-02,Detroit Pistons,DET,9,1.000000,1,0,87,19,18,...,36,8,4,5,16,24,Detroit Pistons,2004-2005,False,0


In [19]:
#populate dfIDtoStat - should be double the size of df
homedf = df[['date', 'homeTeamSubject_id', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_total', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF']]
homedf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']
awaydf = df[['date', 'awayTeamSubject_id', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_total', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF']]
awaydf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']

dfIDtoStat = pd.concat([homedf, awaydf], ignore_index=True)
dfIDtoStat

,date,team_id,team_win%,team_wins,team_losses,team_points_q1,team_points_q2,team_points_q3,team_points_q4,team_points_total,...,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF
0,2024-06-17,2,0.792079,80,21,28,39,19,20,106,...,0.333333,13,15,36,51,25,9,2,7,15
1,2024-06-14,7,0.611650,63,40,34,27,31,30,122,...,0.405405,15,13,39,52,21,7,2,8,17
2,2024-06-12,7,0.607843,62,40,31,20,19,29,99,...,0.360000,9,7,36,43,15,5,1,8,17
3,2024-06-09,2,0.795918,78,20,25,29,29,22,105,...,0.256410,10,10,31,41,29,10,5,10,15
4,2024-06-06,2,0.793814,77,20,37,26,23,21,107,...,0.380952,16,10,37,47,23,6,9,12,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,12,1.000000,1,0,26,20,17,22,109,...,0.235294,4,11,36,47,24,9,5,13,28
51458,2004-11-03,23,1.000000,1,0,27,24,20,27,98,...,0.375000,9,14,27,41,20,9,2,16,26
51459,2004-11-02,8,0.000000,0,1,14,17,26,21,78,...,0.250000,3,16,32,48,17,10,8,16,21
51460,2004-11-02,11,0.000000,0,1,18,20,21,20,79,...,0.461538,6,9,27,36,8,4,5,16,24


In [20]:
windowConditions = {
    'team_win%': False,
    'team_wins': False,
    'team_losses': False,
    'team_points_total': True,
    'team_points_q1': True,
    'team_points_q2': True,
    'team_points_q3': True,
    'team_points_q4': True,
    'team_FG': True,
    'team_FG_made': True,
    'team_3P': True,
    'team_3P_made': True,
    'team_O_Reb': True,
    'team_D_Reb': True,
    'team_Total_Reb': True,
    'team_Ast': True,
    'team_Stl': True,
    'team_Blk': True,
    'team_TO': True,
    'team_PF': True,
}

In [21]:
#showcase of the first iteration
window_size = 20
exX =[]
exY = []
for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfIDtoStat[dfIDtoStat['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
                homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size))
                awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size))
                homeTeam_window = homeTeam_window.drop(columns=['date', 'team_id'])
                #condtional for average or top entry - home Team
                true_cols = [col for col, val in windowConditions.items() if val and col in homeTeam_window.columns]
                false_cols = [col for col, val in windowConditions.items() if not val and col in homeTeam_window.columns]
                homeTeam_window = np.concatenate([homeTeam_window[false_cols].iloc[0].to_numpy(), homeTeam_window[true_cols].mean(axis=0).to_numpy()])
                awayTeam_window = awayTeam_window.drop(columns=['date', 'team_id'])
                #conditional for average or top entry - away Team
                true_cols = [col for col, val in windowConditions.items() if val and col in awayTeam_window.columns]
                false_cols = [col for col, val in windowConditions.items() if not val and col in awayTeam_window.columns]
                awayTeam_window = np.concatenate([awayTeam_window[false_cols].iloc[0].to_numpy(), awayTeam_window[true_cols].mean(axis=0).to_numpy()])
                combined_window = np.hstack((homeTeam_window, awayTeam_window))
                exX.append(combined_window)
                exY.append(row['winner_binary'])
                print(exX)
                print(exY)
                break

[array([  0.79      ,  79.        ,  21.        , 111.2       ,
        30.25      ,  28.3       ,  27.75      ,  24.1       ,
         0.47892073,  40.7       ,   0.36566729,  14.6       ,
         9.75      ,  33.65      ,  43.4       ,  24.5       ,
         6.6       ,   5.8       ,  10.35      ,  15.8       ,
         0.61165049,  63.        ,  40.        , 107.3       ,
        26.6       ,  27.1       ,  26.7       ,  26.9       ,
         0.47368941,  39.45      ,   0.36284279,  12.05      ,
        11.15      ,  32.45      ,  43.6       ,  21.65      ,
         6.35      ,   5.15      ,  12.        ,  19.1       ])]
[0]


In [22]:
windowConditions = {
    'team_win%': 1,
    'team_wins': 1,
    'team_losses': 1,
    'team_points_total': 17,
    'team_points_q1': 17,
    'team_points_q2': 17,
    'team_points_q3': 17,
    'team_points_q4': 20,
    'team_FG': 19,
    'team_FG_made': 17,
    'team_3P': 20,
    'team_3P_made': 15,
    'team_O_Reb': 17,
    'team_D_Reb': 15,
    'team_Total_Reb': 17,
    'team_Ast': 11,
    'team_Stl': 17,
    'team_Blk': 15,
    'team_TO': 15,
    'team_PF': 20,
}

In [27]:
#showcase of the first iteration
window_size = 20
exX =[]
exY = []
for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfIDtoStat[dfIDtoStat['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
                homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size))
                awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size))
                homeTeam_window = homeTeam_window.drop(columns=['date', 'team_id'])
                awayTeam_window = awayTeam_window.drop(columns=['date', 'team_id'])
                #condtional for average or top entry - home Team, away Team
                homeTeam_np = []
                awayTeam_np = []
                for column in homeTeam_window:
                        specified_window = windowConditions.get(column)
                        homeTeam_np.append(homeTeam_window[column].iloc[:specified_window].mean(axis=0))
                        awayTeam_np.append(awayTeam_window[column].iloc[:specified_window].mean(axis=0))       
                combined_window = np.hstack((np.array(homeTeam_np), np.array(awayTeam_np)))
                exX.append(combined_window)
                exY.append(row['winner_binary'])
                print(exX)
                print(exY)
                break

[array([  0.79      ,  79.        ,  21.        ,  30.29411765,
        26.94117647,  27.11764706,  24.1       , 108.64705882,
         0.47679915,  39.41176471,   0.36566729,  14.46666667,
         9.35294118,  33.2       ,  42.52941176,  24.45454545,
         6.11764706,   5.33333333,  10.66666667,  15.8       ,
         0.61165049,  63.        ,  40.        ,  27.94117647,
        26.52941176,  27.11764706,  26.9       , 108.11764706,
         0.47645977,  39.88235294,   0.36284279,  12.06666667,
        11.        ,  33.26666667,  43.76470588,  20.36363636,
         6.        ,   4.93333333,  12.4       ,  19.1       ])]
[0]


In [24]:
#generate the X Y matrices
def df_to_X_y(df, dfWindow, window_size):
    X = []
    y = []
    #each team must have played window_size games before data can be extracted
    for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfWindow[dfWindow['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
            homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            #condtional for average or top entry - home Team
            homeTeam_np = []
            awayTeam_np = []
            for column in homeTeam_window:
                specified_window = windowConditions.get(column)
                homeTeam_np.append(homeTeam_window[column].iloc[:specified_window].mean(axis=0))
                awayTeam_np.append(awayTeam_window[column].iloc[:specified_window].mean(axis=0))       
            combined_window = np.hstack((np.array(homeTeam_np), np.array(awayTeam_np)))
            X.append(combined_window)
            y.append(row['winner_binary'])
    
    return np.array(X), np.array(y)

In [26]:
#X1.Shape is num data, window size, variables
X, y = df_to_X_y(df, dfIDtoStat, 20)
X.shape, y.shape

((25406, 40), (25406,))

In [28]:
#80-10-10 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((20324, 40), (20324,), (2541, 40), (2541,), (2541, 40), (2541,))

In [29]:
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [30]:
#early stopping if model does not improve performance for 50 epochs
earlystopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
#simple model fit
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[earlystopping])

Epoch 1/100
636/636 [==============================] - 2s 1ms/step - loss: 0.6737 - accuracy: 0.6202 - val_loss: 0.6848 - val_accuracy: 0.6088
Epoch 2/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6327 - accuracy: 0.6403 - val_loss: 0.6277 - val_accuracy: 0.6509
Epoch 3/100
636/636 [==============================] - 1s 939us/step - loss: 0.6309 - accuracy: 0.6449 - val_loss: 0.6271 - val_accuracy: 0.6466
Epoch 4/100
636/636 [==============================] - 1s 932us/step - loss: 0.6260 - accuracy: 0.6511 - val_loss: 0.6226 - val_accuracy: 0.6560
Epoch 5/100
636/636 [==============================] - 1s 925us/step - loss: 0.6249 - accuracy: 0.6492 - val_loss: 0.6204 - val_accuracy: 0.6545
Epoch 6/100
636/636 [==============================] - 1s 931us/step - loss: 0.6275 - accuracy: 0.6437 - val_loss: 0.6235 - val_accuracy: 0.6494
Epoch 7/100
636/636 [==============================] - 1s 921us/step - loss: 0.6228 - accuracy: 0.6511 - val_loss: 0.6185 - val_accura

In [32]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
print(accuracy_score(y_test, y_pred_binary))

80/80 [==============================] - 0s 658us/step
0.6654860291223927
